We are interested in how whether the $D_i$ and $q_j$ for our data agrees with our equations : 
    $$D_i = \sum_{j\in J_i} dist(\phi_j,\phi_i)$$
    $$q_{ij} = dist(\phi_{ij},\phi_i)$$

### Object difficulty(D_i)

This just boils down to :

$D_i = \sum 1-\phi_i$

for most of the metrics that compare against ground truth. Num Points, Area ratio excluded. 

In [1]:
%pylab inline
import pandas as pd
from analysis_toolbox import *
from qualityBaseline import *

Populating the interactive namespace from numpy and matplotlib


In [2]:
bb_info = pd.read_csv("computed_my_COCO_BBvals.csv",index_col=0)

Everything that is annotated by worker_id =3 is drawn by me 

In [3]:
metrics_lst = ['Precision [COCO]','Recall [COCO]','Jaccard [COCO]',"NME [COCO]","Num Points",\
               'Precision [Self]','Recall [Self]','Jaccard [Self]',"NME [Self]","Area Ratio"]

In [4]:
bb_info = bb_info.fillna(0)

In [5]:
bb_info[bb_info["object_id"]==12]

,object_id,worker_id,x_locs,y_locs,Num Points,Area Ratio,Jaccard [Self],Precision [Self],Recall [Self],Munkres Euclidean [Self],Jaccard [COCO],Precision [COCO],Recall [COCO],Munkres Euclidean [COCO],NME [COCO],NME [Self]
44,12,8,"[300, 402, 387, 381, 333, 311, 309, 307]","[287, 436, 441, 439, 357, 327, 309, 320]",8,0.007673,0.789606,0.973313,0.807079,443.162096,0.669440,0.974014,0.681614,803.113446,0.963327,0.981807
57,12,6,"[302, 305, 371, 393, 406]","[287, 320, 433, 447, 432]",5,0.012230,0.753149,0.754620,0.997418,469.824399,0.849983,0.871347,0.971962,489.256525,0.977659,0.980712
70,12,23,"[309, 392, 393, 395, 397, 394, 391, 307, 309, ...","[289, 416, 441, 439, 428, 428, 416, 287, 322, ...",12,0.003851,0.399815,0.998061,0.415377,1336.086966,0.359437,1.041399,0.365773,1892.186749,0.913596,0.945149
75,12,22,"[297, 304, 306, 310, 318, 327, 339, 350, 358, ...","[284, 297, 311, 325, 339, 356, 373, 395, 406, ...",21,0.008977,0.833501,0.923173,0.895626,664.172123,0.783449,0.975823,0.798957,466.141428,0.978714,0.972733
80,12,31,"[304, 310, 313, 318, 319, 325, 327, 331, 333, ...","[316, 323, 329, 335, 341, 344, 351, 357, 363, ...",38,0.000784,0.182358,2.192546,0.185887,1014.607754,0.166911,2.345379,0.167812,460.180624,0.978986,0.958347
88,12,42,"[306, 305, 388, 411, 396, 372]","[290, 315, 452, 440, 422, 386]",6,0.010775,0.813585,0.833850,0.970995,715.405201,0.802930,0.898516,0.883008,730.058753,0.966663,0.970630
92,12,46,"[330, 310, 304, 304, 321, 337, 402, 395, 378]","[360, 325, 317, 289, 308, 341, 432, 443, 440]",9,0.010249,0.828522,0.862202,0.954975,850.904800,0.809839,0.926168,0.865729,435.153668,0.980129,0.965067
109,12,53,"[300, 308, 319, 338, 377, 402, 395, 373, 335, ...","[288, 293, 312, 341, 396, 431, 442, 430, 366, ...",10,0.009688,0.887466,0.919253,0.962497,644.260762,0.823705,0.962810,0.850774,442.832171,0.979779,0.973551
119,12,54,"[302, 336, 403, 397, 379, 322, 308]","[287, 338, 430, 441, 438, 346, 322]",7,0.009804,0.909840,0.926036,0.981140,618.231601,0.859363,0.979074,0.875443,446.121695,0.979628,0.974619
216,12,114,"[396, 401, 395, 336, 306, 300, 304, 304, 346, ...","[441, 433, 424, 337, 288, 285, 306, 323, 389, ...",11,0.010682,0.779864,0.817693,0.944000,718.541286,0.842889,0.926814,0.902991,328.744443,0.984988,0.970501


In [11]:
di_oid_lst = []
for oid in list(set(bb_info.object_id)):
#     print 'oid:',oid
    di_lst = []
    object_bbs = bb_info[bb_info["object_id"]==oid]
#     print object_bbs
    for metric in metrics_lst: 
        if metric in ["Num Points", "Area Ratio"]:
            try:
                phi_i = float(object_bbs[object_bbs["worker_id"]==3][metric])
            except(TypeError):
                # some objects don't have worker 3 annotations [this is resolved now]
                print "oid bad"
                pass
        else: 
            phi_i =1
        # Distance calculation (if more than 1d than do something more complicated that subtraction.)
        phi_j = object_bbs[metric]
#         print phi_i
        di = abs(phi_i-phi_j)
        di_lst.append(di)
    di_oid_lst.append([oid,sum(di_lst)])

In [12]:
di_oid_lst,

[[1, 1862.1453103480001],
 [2, 730.80395413700001],
 [3, 550.90493644900005],
 [4, 521.04006969099999],
 [5, 536.51570903200002],
 [6, 124.95696608199999],
 [7, 215.16427766100003],
 [8, 1210.270315862],
 [9, 241.458978247],
 [10, 695.58672663599998],
 [11, 280.25155263400001],
 [12, 302.58229363199996],
 [13, 411.605881802],
 [14, 587.51461282699995],
 [15, 209.20538169000002],
 [16, 187.56547472599999],
 [17, 638.92980692900005],
 [18, 465.67019910800002],
 [19, 259.78862319000001],
 [20, 665.82724655499999],
 [21, 456.58647585199998],
 [22, 487.20016268699999],
 [23, 334.37911144200001],
 [24, 543.20811989200001],
 [25, 990.395920596],
 [26, 833.02087591700001],
 [27, 667.26778207899997],
 [28, 1315.1888362689999],
 [29, 982.43482151800004],
 [30, 1530.1758597380001],
 [31, 703.49943947500014],
 [32, 764.07215588099996],
 [33, 659.19380514099998],
 [34, 1020.891756053],
 [35, 1548.5267612839998],
 [36, 331.78305404299999],
 [37, 640.19120912699998],
 [38, 447.29284637900003],
 [39, 

We can conduct separate analysis based on 3  different errors

In [13]:
task_ambiguity = [1,4,8,10,18,20,22,27,28,29,30,31,32,33,35,40,42]
small_area = [1,15,22]
high_numPts=[5,7,9,10,11,12,28,29,30,37]

had to manually some of the missing self annotated box values that I added in later

```
compute_img_to_bb_area_ratio("COCO_train2014_000000000307",[385,396,416,426,430,415,394],[59,52,48,52,59,65,61])

compute_img_to_bb_area_ratio("COCO_train2014_000000000307",[482,469,466,454,445,450,438,435,427,409,410,412,423,443,451,441,439,439,443,453,455,458,469,482,485,495,511,519,527,535,547,554,556,552,561,567,580,577,572,570,547,540,534,527,515,500],[200,212,232,243,231,217,226,219,233,224,245,260,276,293,313,323,343,360,366,358,350,344,352,351,365,366,365,359,358,357,359,349,343,338,333,323,297,283,246,233,217,213,218,206,205,216])

compute_img_to_bb_area_ratio("COCO_train2014_000000000127",[304,307,308,381,394,402,402,310],[292,311,323,439,443,440,431,301])

compute_img_to_bb_area_ratio("COCO_train2014_000000000127",[393,411,440,472,520,541,539,537,537,548,561,564,566,559,550,535,515,494,478,456,442,430,421,406,400,392,387,384],[237,228,224,225,238,257,270,278,285,286,293,299,312,325,331,336,337,334,345,346,344,340,335,319,312,294,271,250])
```

In [29]:
di_oid_lst

[[1, -1285.0243432944617],
 [2, -41.934912509337387],
 [3, 178.90140456009431],
 [4, 332.97516409827557],
 [5, 220.50935487596368],
 [6, -1.3855763635579414],
 [7, 121.14546580290758],
 [8, 271.55825037271285],
 [9, -20.59727499715742]]

In [8]:
bb_info

,object_id,worker_id,x_locs,y_locs,Num Points,Area Ratio,Jaccard [Self],Precision [Self],Recall [Self],Munkres Euclidean [Self],Jaccard [COCO],Precision [COCO],Recall [COCO],Munkres Euclidean [COCO],NME [COCO],NME [Self]
0,8,3,"[302, 264, 254, 250, 250, 250, 248, 247, 247, ...","[124, 131, 137, 145, 152, 160, 167, 173, 188, ...",45,0.069980,1.000000,1.000000,1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000
1,11,3,"[347, 383, 385, 380, 393, 414, 429, 429, 408, ...","[278, 321, 332, 341, 351, 357, 356, 338, 333, ...",13,0.004887,1.000000,1.000000,1.000000,0.000000,0.823483,0.925719,0.881747,223.430073,0.989797,1.000000
2,16,3,"[354, 353, 353, 361, 378, 390, 402, 409, 413, ...","[455, 434, 405, 381, 370, 370, 381, 393, 413, ...",13,0.019753,1.000000,1.000000,1.000000,0.000000,0.866697,0.928287,0.928892,321.992822,0.985297,1.000000
3,9,3,"[258, 258, 291, 303, 355, 349, 357, 363, 388, ...","[218, 270, 275, 289, 285, 279, 278, 283, 281, ...",13,0.029879,1.000000,1.000000,1.000000,0.000000,0.942081,0.966382,0.974001,361.914598,0.983474,1.000000
4,13,3,"[243, 181, 190, 224, 238, 247, 254, 268, 272, ...","[273, 345, 376, 398, 399, 400, 400, 404, 385, ...",12,0.036020,1.000000,1.000000,1.000000,0.000000,0.900818,0.966149,0.930176,407.973728,0.981370,1.000000
5,5,3,"[432, 437, 443, 456, 454, 474, 483, 483, 486, ...","[208, 195, 173, 168, 136, 130, 136, 142, 151, ...",20,0.012716,1.000000,1.000000,1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000
6,43,3,"[103, 94, 89, 96, 106, 118, 128, 136, 125, 137...","[432, 430, 441, 441, 446, 453, 453, 449, 476, ...",38,0.052991,1.000000,1.000000,1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000
7,4,3,"[91, 96, 93, 88, 85, 88, 108, 124, 134, 134, 1...","[335, 355, 381, 418, 444, 455, 462, 452, 442, ...",14,0.010319,1.000000,1.000000,1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000
8,37,3,"[522, 522, 517, 517, 528, 536, 544, 544, 541, ...","[225, 217, 210, 199, 192, 195, 204, 211, 218, ...",47,0.033953,1.000000,1.000000,1.000000,0.000000,0.112746,0.113849,0.920877,2297.126982,0.895104,1.000000
9,3,3,"[47, 90, 97, 90, 86, 47, 52]","[319, 340, 360, 409, 422, 407, 340]",7,0.011981,1.000000,1.000000,1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000
